In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from math import log2
from scipy.stats import zscore, entropy
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor

ModuleNotFoundError: No module named 'xgboost'

In [3]:
df = pd.read_csv(r'C:\Users\Cameron\OneDrive\Documents\DCU_Data\MWBW full dataset 2377 participants_anonymised (1).csv')

#drop all nan values for the whole dataset
drop_set = df[['new_calc_bmi','VO2max','age','puntkick_total','handpass_total','groundstrike_total','balance_catch_total','run_total','skip_total','gallop_total','slide_total','hop_total','hj_total','vj_total','mnm_total','catch_total','throw_total','roll_total','kick_total','2hs_total','1hs_total','bounce_total','balance_total']].copy()
drop_set = drop_set.dropna()
drop_set = drop_set.reset_index(drop=True)

drop_set.shape[0]

913

In [3]:
drop_set = drop_set.astype('float')
drop_set = drop_set[(drop_set['new_calc_bmi'] >= 10) & (drop_set['new_calc_bmi'] <= 40)]
drop_set = drop_set.reset_index(drop=True)

drop_set

,new_calc_bmi,VO2max,age,puntkick_total,handpass_total,groundstrike_total,balance_catch_total,run_total,skip_total,gallop_total,...,vj_total,mnm_total,catch_total,throw_total,roll_total,kick_total,2hs_total,1hs_total,bounce_total,balance_total
0,20.087236,46.29,10.0,10.0,10.0,10.0,7.0,7.0,4.0,2.0,...,12.0,9.0,6.0,7.0,6.0,6.0,6.0,8.0,5.0,8.0
1,15.996408,61.86,9.0,10.0,6.0,7.0,5.0,8.0,6.0,4.0,...,11.0,9.0,6.0,4.0,5.0,6.0,7.0,8.0,6.0,8.0
2,15.254607,54.93,9.0,2.0,4.0,1.0,5.0,8.0,6.0,2.0,...,12.0,8.0,6.0,3.0,7.0,7.0,6.0,1.0,5.0,8.0
3,20.698502,45.69,10.0,6.0,6.0,8.0,4.0,8.0,4.0,2.0,...,10.0,6.0,6.0,4.0,5.0,3.0,5.0,8.0,0.0,8.0
4,15.472683,59.55,10.0,10.0,10.0,10.0,8.0,8.0,6.0,2.0,...,10.0,9.0,6.0,5.0,4.0,8.0,10.0,8.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,14.367644,61.82,12.0,10.0,10.0,0.0,0.0,8.0,5.0,6.0,...,12.0,12.0,6.0,7.0,7.0,8.0,10.0,7.0,4.0,8.0
908,22.031726,53.02,12.0,8.0,10.0,0.0,0.0,8.0,6.0,5.0,...,10.0,6.0,6.0,4.0,5.0,4.0,5.0,4.0,1.0,8.0
909,17.777778,51.62,13.0,8.0,10.0,0.0,0.0,8.0,5.0,6.0,...,12.0,8.0,6.0,4.0,7.0,2.0,5.0,3.0,5.0,8.0
910,21.113276,55.56,12.0,10.0,10.0,0.0,0.0,8.0,6.0,6.0,...,12.0,12.0,6.0,7.0,8.0,8.0,7.0,8.0,6.0,8.0


In [4]:
z_score_list = ['new_calc_bmi','VO2max','age']
z_set = zscore(drop_set[z_score_list].copy())

In [5]:
y_var = z_set['VO2max'].copy()

In [6]:
x_var1 = z_set[['new_calc_bmi','age']].copy()

In [7]:
test = pd.concat([x_var1,y_var],axis=1)

In [8]:
x_var2_list = ['puntkick_total','handpass_total','groundstrike_total','balance_catch_total','run_total','skip_total','gallop_total','slide_total','hop_total','hj_total','vj_total','mnm_total','catch_total','throw_total','roll_total','kick_total','2hs_total','1hs_total','bounce_total','balance_total']
x_var2 = pd.get_dummies(drop_set[x_var2_list].copy(),columns=x_var2_list)

In [9]:
x_var = pd.concat([x_var1,x_var2],axis=1)

In [10]:
column_names = x_var.columns

X_train, X_test, y_train, y_test = train_test_split(x_var, y_var, test_size=0.25)

x_train = pd.DataFrame(X_train, columns=column_names)
y_train = pd.DataFrame(y_train, columns=['VO2max'])
X_test = pd.DataFrame(X_test, columns=column_names)
y_test = pd.DataFrame(y_test, columns=['VO2max'])

In [11]:
model = LinearRegression().fit(x_train,y_train)

y_hat = model.predict(X_test)

In [12]:
mean_absolute = mean_absolute_error(y_test, y_hat)

print('Linear Regression\n'
      '-------------------------')

print(f'The mean absolute error is: {mean_absolute}')

r2 = r2_score(y_test, y_hat)

print(f'The R2 score for the model is: {r2}')

Linear Regression
-------------------------
The mean absolute error is: 9493542145.54416
The R2 score for the model is: -7.41357667778567e+21


In [13]:
numpy_y_train = y_train.to_numpy(dtype=float, copy=True)

numpy_y_train = np.ravel(numpy_y_train)

model = SVR(kernel='rbf').fit(X_train, numpy_y_train)

y_hat = model.predict(X_test)

In [14]:
mean_absolute = mean_absolute_error(y_test, y_hat)

print('Support Vector Regression\n'
      '-------------------------')

print(f'The mean absolute error is: {mean_absolute}')

r2 = r2_score(y_test, y_hat)

print(f'The R2 score for the model is: {r2}')

Support Vector Regression
-------------------------
The mean absolute error is: 0.6958377965314034
The R2 score for the model is: 0.2482319094228418


In [15]:
model = GradientBoostingRegressor().fit(X_train, numpy_y_train)

y_hat = model.predict(X_test)

In [16]:
print('Gradient Boosting Regressor\n'
      '---------------------------')

mean_absolute = mean_absolute_error(y_test, y_hat)

print(f'The mean absolute error is: {mean_absolute}')

r2 = r2_score(y_test, y_hat)

print(f'The R2 score for the model is: {r2}')

Gradient Boosting Regressor
---------------------------
The mean absolute error is: 0.6927285256617166
The R2 score for the model is: 0.2628549923910509


In [17]:
model = xgb.XGBRegressor().fit(X_train, numpy_y_train)

y_hat = model.predict(X_test)

C:\Users\Cameron\anaconda3\envs\DCU_ENV\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [18]:
print('XGB Regressor\n'
      '---------------------------')

mean_absolute = mean_absolute_error(y_test, y_hat)

print(f'The mean absolute error is: {mean_absolute}')

r2 = r2_score(y_test, y_hat)

print(f'The R2 score for the model is: {r2}')

XGB Regressor
---------------------------
The mean absolute error is: 0.7432136576817527
The R2 score for the model is: 0.13779494221978494


In [19]:
drop_set2 = df[['gender','new_calc_bmi','VO2max','age','puntkick_total','handpass_total','groundstrike_total','balance_catch_total','run_total','skip_total','gallop_total','slide_total','hop_total','hj_total','vj_total','mnm_total','catch_total','throw_total','roll_total','kick_total','2hs_total','1hs_total','bounce_total','balance_total']].copy()
drop_set2 = drop_set2.dropna()
drop_set2 = drop_set2.reset_index(drop=True)

drop_set2

,gender,new_calc_bmi,VO2max,age,puntkick_total,handpass_total,groundstrike_total,balance_catch_total,run_total,skip_total,...,vj_total,mnm_total,catch_total,throw_total,roll_total,kick_total,2hs_total,1hs_total,bounce_total,balance_total
0,1.0,20.087236,46.29,10.0,10.0,10.0,10.0,7.0,7.0,4.0,...,12.0,9.0,6.0,7.0,6.0,6.0,6.0,8.0,5.0,8.0
1,1.0,15.996408,61.86,9.0,10.0,6.0,7.0,5.0,8.0,6.0,...,11.0,9.0,6.0,4.0,5.0,6.0,7.0,8.0,6.0,8.0
2,1.0,15.254607,54.93,9.0,2.0,4.0,1.0,5.0,8.0,6.0,...,12.0,8.0,6.0,3.0,7.0,7.0,6.0,1.0,5.0,8.0
3,2.0,20.698502,45.69,10.0,6.0,6.0,8.0,4.0,8.0,4.0,...,10.0,6.0,6.0,4.0,5.0,3.0,5.0,8.0,0.0,8.0
4,1.0,15.472683,59.55,10.0,10.0,10.0,10.0,8.0,8.0,6.0,...,10.0,9.0,6.0,5.0,4.0,8.0,10.0,8.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,1.0,14.367644,61.82,12.0,10.0,10.0,0.0,0.0,8.0,5.0,...,12.0,12.0,6.0,7.0,7.0,8.0,10.0,7.0,4.0,8.0
909,2.0,22.031726,53.02,12.0,8.0,10.0,0.0,0.0,8.0,6.0,...,10.0,6.0,6.0,4.0,5.0,4.0,5.0,4.0,1.0,8.0
910,2.0,17.777778,51.62,13.0,8.0,10.0,0.0,0.0,8.0,5.0,...,12.0,8.0,6.0,4.0,7.0,2.0,5.0,3.0,5.0,8.0
911,1.0,21.113276,55.56,12.0,10.0,10.0,0.0,0.0,8.0,6.0,...,12.0,12.0,6.0,7.0,8.0,8.0,7.0,8.0,6.0,8.0


In [20]:
male_set = drop_set2[(drop_set2['gender'] == 1)]

male_set

,gender,new_calc_bmi,VO2max,age,puntkick_total,handpass_total,groundstrike_total,balance_catch_total,run_total,skip_total,...,vj_total,mnm_total,catch_total,throw_total,roll_total,kick_total,2hs_total,1hs_total,bounce_total,balance_total
0,1.0,20.087236,46.29,10.0,10.0,10.0,10.0,7.0,7.0,4.0,...,12.0,9.0,6.0,7.0,6.0,6.0,6.0,8.0,5.0,8.0
1,1.0,15.996408,61.86,9.0,10.0,6.0,7.0,5.0,8.0,6.0,...,11.0,9.0,6.0,4.0,5.0,6.0,7.0,8.0,6.0,8.0
2,1.0,15.254607,54.93,9.0,2.0,4.0,1.0,5.0,8.0,6.0,...,12.0,8.0,6.0,3.0,7.0,7.0,6.0,1.0,5.0,8.0
4,1.0,15.472683,59.55,10.0,10.0,10.0,10.0,8.0,8.0,6.0,...,10.0,9.0,6.0,5.0,4.0,8.0,10.0,8.0,0.0,6.0
5,1.0,17.013714,52.62,9.0,5.0,0.0,10.0,6.0,6.0,2.0,...,8.0,5.0,6.0,7.0,6.0,8.0,4.0,4.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,1.0,18.335764,55.83,10.0,10.0,10.0,0.0,0.0,8.0,6.0,...,12.0,10.0,6.0,6.0,8.0,8.0,8.0,3.0,6.0,8.0
902,1.0,17.406665,56.89,11.0,10.0,9.0,0.0,0.0,8.0,6.0,...,12.0,12.0,6.0,7.0,7.0,8.0,8.0,8.0,6.0,8.0
906,1.0,18.510806,56.89,12.0,10.0,10.0,0.0,0.0,8.0,4.0,...,12.0,12.0,5.0,7.0,7.0,8.0,9.0,2.0,6.0,6.0
908,1.0,14.367644,61.82,12.0,10.0,10.0,0.0,0.0,8.0,5.0,...,12.0,12.0,6.0,7.0,7.0,8.0,10.0,7.0,4.0,8.0


In [21]:
female_set = drop_set2[(drop_set2['gender'] == 2)]

female_set

,gender,new_calc_bmi,VO2max,age,puntkick_total,handpass_total,groundstrike_total,balance_catch_total,run_total,skip_total,...,vj_total,mnm_total,catch_total,throw_total,roll_total,kick_total,2hs_total,1hs_total,bounce_total,balance_total
3,2.0,20.698502,45.69,10.0,6.0,6.0,8.0,4.0,8.0,4.0,...,10.0,6.0,6.0,4.0,5.0,3.0,5.0,8.0,0.0,8.0
7,2.0,16.693342,61.86,9.0,4.0,0.0,6.0,6.0,8.0,6.0,...,6.0,7.0,5.0,6.0,2.0,8.0,4.0,5.0,6.0,7.0
8,2.0,16.247184,64.17,10.0,10.0,5.0,10.0,2.0,8.0,5.0,...,8.0,9.0,6.0,6.0,4.0,8.0,5.0,7.0,6.0,5.0
10,2.0,14.986723,57.24,9.0,10.0,7.0,7.0,4.0,7.0,4.0,...,11.0,6.0,4.0,5.0,4.0,6.0,9.0,2.0,6.0,8.0
12,2.0,16.856738,52.62,9.0,10.0,5.0,5.0,0.0,8.0,4.0,...,10.0,6.0,6.0,3.0,6.0,8.0,6.0,4.0,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,2.0,14.112573,49.50,11.0,6.0,7.0,0.0,0.0,4.0,5.0,...,12.0,7.0,6.0,4.0,4.0,8.0,4.0,4.0,6.0,8.0
907,2.0,19.173217,58.10,12.0,10.0,10.0,0.0,0.0,8.0,5.0,...,12.0,9.0,6.0,6.0,4.0,8.0,6.0,4.0,6.0,8.0
909,2.0,22.031726,53.02,12.0,8.0,10.0,0.0,0.0,8.0,6.0,...,10.0,6.0,6.0,4.0,5.0,4.0,5.0,4.0,1.0,8.0
910,2.0,17.777778,51.62,13.0,8.0,10.0,0.0,0.0,8.0,5.0,...,12.0,8.0,6.0,4.0,7.0,2.0,5.0,3.0,5.0,8.0


In [24]:
#VO2max is for age 9 to 13 for both genders

male_9 = male_set[(male_set['age'] == 9)]
male_10 = male_set[(male_set['age'] == 10)]
male_11 = male_set[(male_set['age'] == 11)]
male_12 = male_set[(male_set['age'] == 12)]
male_13 = male_set[(male_set['age'] == 13)]

print(f'male_9 size: {male_9.shape}')
print(f'male_10 size: {male_10.shape}')
print(f'male_11 size: {male_11.shape}')
print(f'male_12 size: {male_12.shape}')
print(f'male_13 size: {male_13.shape}')

male_9 size: (56, 24)
male_10 size: (140, 24)
male_11 size: (113, 24)
male_12 size: (115, 24)
male_13 size: (58, 24)


In [25]:
female_9 = female_set[(female_set['age'] == 9)]
female_10 = female_set[(female_set['age'] == 10)]
female_11 = female_set[(female_set['age'] == 11)]
female_12 = female_set[(female_set['age'] == 12)]
female_13 = female_set[(female_set['age'] == 13)]

print(f'female_9 size: {female_9.shape}')
print(f'female_10 size: {female_10.shape}')
print(f'female_11 size: {female_11.shape}')
print(f'female_12 size: {female_12.shape}')
print(f'female_13 size: {female_13.shape}')

female_9 size: (62, 24)
female_10 size: (135, 24)
female_11 size: (107, 24)
female_12 size: (82, 24)
female_13 size: (45, 24)


In [28]:
# Men
# Very Poor: <35.0
# Poor: 35.0 - 38.3
# Fair: 38.4 - 45.1
# Good: 45.2 - 50.9
# Excellent: 51.0 - 55.9
# Superior: >55.9

# Women
# Very Poor: <25.0
# Poor: 25.0 - 30.9
# Fair: 31.0 - 34.9 
# Good: 35.0 - 38.9
# Excellent: 39.0 - 41.9
# Superior: >41.9

In [69]:
male9_VeryPoor = male_9[(male_9['VO2max'] < 35.0)]
male9_Poor = male_9[(male_9['VO2max'] > 35.0) & (male_9['VO2max'] < 38.3)]
male9_Fair = male_9[(male_9['VO2max'] > 38.4) & (male_9['VO2max'] < 45.1)]
male9_Good = male_9[(male_9['VO2max'] > 45.2) & (male_9['VO2max'] < 50.9)]
male9_Excellent = male_9[(male_9['VO2max'] > 51.0) & (male_9['VO2max'] < 55.9)]
male9_Superior = male_9[(male_9['VO2max'] > 55.9)]

print(f'male_9 rows: {male_9.shape[0]}')
print(f'male9_VeryPoor shape: {male9_VeryPoor.shape}')
print(f'male9_Poor shape: {male9_Poor.shape}')
print(f'male9_Fair shape: {male9_Fair.shape}')
print(f'male9_Good shape: {male9_Good.shape}')
print(f'male9_Excellent shape: {male9_Excellent.shape}')
print(f'male9_Superior shape: {male9_Superior.shape}')

male_9 rows: 56
male9_VeryPoor shape: (0, 24)
male9_Poor shape: (0, 24)
male9_Fair shape: (10, 24)
male9_Good shape: (31, 24)
male9_Excellent shape: (10, 24)
male9_Superior shape: (5, 24)


In [71]:
#entropy for male_9

e = entropy([10/56,31/56,10/56,5/56],base=2)

print(e)

1.6711354935901546


In [67]:
male10_VeryPoor = male_10[(male_10['VO2max'] < 35.0)]
male10_Poor = male_10[(male_10['VO2max'] > 35.0) & (male_10['VO2max'] < 38.3)]
male10_Fair = male_10[(male_10['VO2max'] > 38.4) & (male_10['VO2max'] < 45.1)]
male10_Good = male_10[(male_10['VO2max'] > 45.2) & (male_10['VO2max'] < 50.9)]
male10_Excellent = male_10[(male_10['VO2max'] > 51.0) & (male_10['VO2max'] < 55.9)]
male10_Superior = male_10[(male_10['VO2max'] > 55.9)]

print(f'male_10 rows: {male_10.shape[0]}')
print(f'male10_VeryPoor shape: {male10_VeryPoor.shape}')
print(f'male10_Poor shape: {male10_Poor.shape}')
print(f'male10_Fair shape: {male10_Fair.shape}')
print(f'male10_Good shape: {male10_Good.shape}')
print(f'male10_Excellent shape: {male10_Excellent.shape}')
print(f'male10_Superior shape: {male10_Superior.shape}')

male_10 rows: 140
male10_VeryPoor shape: (0, 24)
male10_Poor shape: (1, 24)
male10_Fair shape: (21, 24)
male10_Good shape: (62, 24)
male10_Excellent shape: (40, 24)
male10_Superior shape: (16, 24)


In [68]:
#entropy for male_10

e = entropy([1/140,21/140,62/140,40/140,16/140],base=2)

print(e)

1.85588329610329


In [65]:
male11_VeryPoor = male_11[(male_11['VO2max'] < 35.0)]
male11_Poor = male_11[(male_11['VO2max'] > 35.0) & (male_11['VO2max'] < 38.3)]
male11_Fair = male_11[(male_11['VO2max'] > 38.4) & (male_11['VO2max'] < 45.1)]
male11_Good = male_11[(male_11['VO2max'] > 45.2) & (male_11['VO2max'] < 50.9)]
male11_Excellent = male_11[(male_11['VO2max'] > 51.0) & (male_11['VO2max'] < 55.9)]
male11_Superior = male_11[(male_11['VO2max'] > 55.9)]

print(f'male_11 rows: {male_11.shape[0]}')
print(f'male11_VeryPoor shape: {male11_VeryPoor.shape}')
print(f'male11_Poor shape: {male11_Poor.shape}')
print(f'male11_Fair shape: {male11_Fair.shape}')
print(f'male11_Good shape: {male11_Good.shape}')
print(f'male11_Excellent shape: {male11_Excellent.shape}')
print(f'male11_Superior shape: {male11_Superior.shape}')

male_11 rows: 113
male11_VeryPoor shape: (1, 24)
male11_Poor shape: (0, 24)
male11_Fair shape: (31, 24)
male11_Good shape: (38, 24)
male11_Excellent shape: (30, 24)
male11_Superior shape: (13, 24)


In [66]:
#entropy for male_11

e = entropy([1/113,31/113,38/113,30/113,13/113],base=2)

print(e)

1.9678447558936027


In [62]:
male12_VeryPoor = male_12[(male_12['VO2max'] < 35.0)]
male12_Poor = male_12[(male_12['VO2max'] > 35.0) & (male_12['VO2max'] < 38.3)]
male12_Fair = male_12[(male_12['VO2max'] > 38.4) & (male_12['VO2max'] < 45.1)]
male12_Good = male_12[(male_12['VO2max'] > 45.2) & (male_12['VO2max'] < 50.9)]
male12_Excellent = male_12[(male_12['VO2max'] > 51.0) & (male_12['VO2max'] < 55.9)]
male12_Superior = male_12[(male_12['VO2max'] > 55.9)]

print(f'male12 rows: {male_12.shape[0]}')
print(f'male12_VeryPoor shape: {male12_VeryPoor.shape}')
print(f'male12_Poor shape: {male12_Poor.shape}')
print(f'male12_Fair shape: {male12_Fair.shape}')
print(f'male12_Good shape: {male12_Good.shape}')
print(f'male12_Excellent shape: {male12_Excellent.shape}')
print(f'male12_Superior shape: {male12_Superior.shape}')

male12 rows: 115
male12_VeryPoor shape: (0, 24)
male12_Poor shape: (4, 24)
male12_Fair shape: (24, 24)
male12_Good shape: (35, 24)
male12_Excellent shape: (36, 24)
male12_Superior shape: (16, 24)


In [79]:
#entropy for male_12

e = entropy([4/115,24/115,35/115,36/115,16/115],base=2)

print(e)

2.0830439147504207


In [64]:
male13_VeryPoor = male_13[(male_13['VO2max'] < 35.0)]
male13_Poor = male_13[(male_13['VO2max'] > 35.0) & (male_13['VO2max'] < 38.3)]
male13_Fair = male_13[(male_13['VO2max'] > 38.4) & (male_13['VO2max'] < 45.1)]
male13_Good = male_13[(male_13['VO2max'] > 45.2) & (male_13['VO2max'] < 50.9)]
male13_Excellent = male_13[(male_13['VO2max'] > 51.0) & (male_13['VO2max'] < 55.9)]
male13_Superior = male_13[(male_13['VO2max'] > 55.9)]

print(f'male_13 rows: {male_13.shape[0]}')
print(f'male13_VeryPoor shape: {male13_VeryPoor.shape}')
print(f'male13_Poor shape: {male13_Poor.shape}')
print(f'male13_Fair shape: {male13_Fair.shape}')
print(f'male13_Good shape: {male13_Good.shape}')
print(f'male13_Excellent shape: {male13_Excellent.shape}')
print(f'male13_Superior shape: {male13_Superior.shape}')

male_13 rows: 58
male13_VeryPoor shape: (0, 24)
male13_Poor shape: (2, 24)
male13_Fair shape: (9, 24)
male13_Good shape: (14, 24)
male13_Excellent shape: (18, 24)
male13_Superior shape: (15, 24)


In [59]:
#entropy for male_13

e = entropy([2/58,9/58,14/58,18/58,15/58],base=2)

print(e)

2.1080792213021122


In [82]:
female9_VeryPoor = female_9[(female_9['VO2max'] < 25.0)]
female9_Poor = female_9[(female_9['VO2max'] > 25.0) & (female_9['VO2max'] < 30.9)]
female9_Fair = female_9[(female_9['VO2max'] > 31.0) & (female_9['VO2max'] < 34.9)]
female9_Good = female_9[(female_9['VO2max'] > 35.0) & (female_9['VO2max'] < 38.9)]
female9_Excellent = female_9[(female_9['VO2max'] > 39.0) & (female_9['VO2max'] < 41.9)]
female9_Superior = female_9[(female_9['VO2max'] > 41.9)]

print(f'female_9 rows: {female_9.shape[0]}')
print(f'female9_VeryPoor shape: {female9_VeryPoor.shape}')
print(f'female9_Poor shape: {female9_Poor.shape}')
print(f'female9_Fair shape: {female9_Fair.shape}')
print(f'female9_Good shape: {female9_Good.shape}')
print(f'female9_Excellent shape: {female9_Excellent.shape}')
print(f'female9_Superior shape: {female9_Superior.shape}')

female_9 rows: 62
female9_VeryPoor shape: (0, 24)
female9_Poor shape: (0, 24)
female9_Fair shape: (0, 24)
female9_Good shape: (0, 24)
female9_Excellent shape: (0, 24)
female9_Superior shape: (62, 24)


In [ ]:
female9_VeryPoor = male_9[(male_9['VO2max'] < 25.0)]
female9_Poor = male_9[(male_9['VO2max'] > 25.0) & (male_9['VO2max'] < 30.9)]
female9_Fair = male_9[(male_9['VO2max'] > 31.0) & (male_9['VO2max'] < 34.9)]
female9_Good = male_9[(male_9['VO2max'] > 35.0) & (male_9['VO2max'] < 38.9)]
female9_Excellent = male_9[(male_9['VO2max'] > 39.0) & (male_9['VO2max'] < 41.9)]
female9_Superior = male_9[(male_9['VO2max'] > 41.9)]

print(f'female_9 rows: {female_9.shape[0]}')
print(f'female9_VeryPoor shape: {female9_VeryPoor.shape}')
print(f'female9_Poor shape: {female9_Poor.shape}')
print(f'female9_Fair shape: {female9_Fair.shape}')
print(f'female9_Good shape: {female9_Good.shape}')
print(f'female9_Excellent shape: {female9_Excellent.shape}')
print(f'female9_Superior shape: {female9_Superior.shape}')